In [1]:
import vamp
import librosa
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")
import essentia.standard as es
import numpy as np
import spleeter
import pydub

import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
from mido import MidiFile,MidiTrack
from midi2audio import FluidSynth
from collections import Counter
from scipy.stats.stats import pearsonr
import os
import numpy
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.io import wavfile


In [2]:
class Melodia:
    
    def melodia_plugin(self,audio_file):
        # This is how we load audio using Essentia
        loader = es.MonoLoader(filename=audio_file, downmix = 'mix', sampleRate = 44100)
        audio = loader()
        # This is how we load audio using Librosa
        audio, sr = librosa.load(audio_file, sr=44100, mono=True)
#         print(sr)
        data = vamp.collect(audio, sr, "mtg-melodia:melodia")
        # parameter values are specified by providing a dicionary to the optional "parameters" parameter:
        params = {"minfqr": 100.0, "maxfqr": 800.0, "voicing": 0.2, "minpeaksalience": 0.0}

        data = vamp.collect(audio, sr, "mtg-melodia:melodia", parameters=params)
        hop, melody = data['vector']
#         print(len(melody))
        timestamps = 8 * 128/44100.0 + np.arange(len(melody)) * (128/44100.0)
        melody_pos = melody[:]
        melody_pos[melody<=0] = 0
        melody_data = []
#         self.plot_melody(timestamps,melody,audio_file)
        return timestamps,melody_pos,audio_file
    
    def plot_melody(self,timestamps,melody,audio_file):
        melody_pos = melody[:]
        melody_pos[melody<=0] = None
        plt.figure(figsize=(18,6))
        plt.plot(timestamps, melody_pos)
        plt.title(audio_file)
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        return plt.show()

In [3]:
file_path = "/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/"

In [4]:
melodia = Melodia()
arr=[]
for file in os.listdir(file_path):
    if not file.startswith('.'):
        file_name = os.path.splitext(file)[0]
        
        path = file_path +file_name+"/"
        for file in os.listdir(path):
            if not file.startswith('.'):
                print(file)
                
                timestamps,melody_pos,audio_file=melodia.melodia_plugin(path+file)
                arr.append([timestamps,melody_pos,file])
                print(audio_file)
                
                
# 

MusicDelta_Britpop_MIX.wav
/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/MusicDelta_Britpop/MusicDelta_Britpop_MIX.wav
KarimDouaidy_Hopscotch_MIX.wav
/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/KarimDouaidy_Hopscotch/KarimDouaidy_Hopscotch_MIX.wav
AlexanderRoss_VelvetCurtain_MIX.wav
/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/AlexanderRoss_VelvetCurtain/AlexanderRoss_VelvetCurtain_MIX.wav
StrandOfOaks_Spacestation_MIX.wav
/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/StrandOfOaks_Spacestation/StrandOfOaks_Spacestation_MIX.wav
MatthewEntwistle_Lontano_MIX.wav
/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/MatthewEntwistle_Lontano/MatthewEntwistle_Lontano_MIX.wav
DreamersOfTheGhetto_HeavyLove_MIX.wav
/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/V1/DreamersOfTheGhetto_HeavyLove/DreamersOfTheGhetto_HeavyLove_MIX.wav
Mozart_BesterJungling_MIX

KeyboardInterrupt: 

In [5]:
print(arr[0])

[[array([2.32199546e-02, 2.61224490e-02, 2.90249433e-02, ...,
       3.68036281e+01, 3.68065306e+01, 3.68094331e+01]), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'MusicDelta_Britpop_MIX.wav'], [array([2.32199546e-02, 2.61224490e-02, 2.90249433e-02, ...,
       2.94016871e+02, 2.94019773e+02, 2.94022676e+02]), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'KarimDouaidy_Hopscotch_MIX.wav'], [array([2.32199546e-02, 2.61224490e-02, 2.90249433e-02, ...,
       5.14504853e+02, 5.14507755e+02, 5.14510658e+02]), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'AlexanderRoss_VelvetCurtain_MIX.wav'], [array([2.32199546e-02, 2.61224490e-02, 2.90249433e-02, ...,
       2.43876281e+02, 2.43879184e+02, 2.43882086e+02]), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'StrandOfOaks_Spacestation_MIX.wav'], [array([2.32199546e-02, 2.61224490e-02, 2.90249433e-02, ...,
       2.98016508e+02, 2.98019410e+02, 2.98022313e+02]), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 

In [6]:
file_path_melody = "/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/Melody2"

In [93]:
import re
import mir_eval
import csv
import pandas as pd
from statistics import mean

In [90]:
output_arr=[]
for ground_truth in os.listdir(file_path_melody):
    for melody_extracted in arr:
        
        if not ground_truth.startswith('.'):
            ground_truth_filename = os.path.splitext(ground_truth)[0]
            melody_extracted_renamed = re.sub(r'_MIX.wav', '', melody_extracted[2], re.I)
            ground_truth_renamed = re.sub(r'_MELODY1', '', ground_truth_filename, re.I)
            
            
            timestamps_extracted_melody = melody_extracted[0]
            freq_extracted_melody=melody_extracted[1]
            
            if melody_extracted_renamed==ground_truth_renamed:
                path = file_path_melody +ground_truth

                df = pd.read_csv(path)
                df.columns =['Timestamps', 'Freq']
                
                timestamps_ground_truth = df['Timestamps'].tolist()
                melody_ground_truth = df['Freq'].tolist()
                
                (ref_v, ref_c,est_v, est_c) = mir_eval.melody.to_cent_voicing(timestamps_ground_truth,
                                                     melody_ground_truth,
                                                    timestamps_extracted_melody,
                                                    freq_extracted_melody)
#   Voicing Recall Rate              
                voicing_recall_rate = mir_eval.melody.voicing_recall(ref_v, est_v)
#     Voicing False Alarm Rate
                voicing_false_alarm_rate=mir_eval.melody.voicing_false_alarm(ref_v, est_v)
#   Raw Pitch Accuracy  
                raw_pitch = mir_eval.melody.raw_pitch_accuracy(ref_v, ref_c,est_v, est_c)
#     Raw Chroma Accuracy
                raw_chroma = mir_eval.melody.raw_chroma_accuracy(ref_v, ref_c,est_v, est_c)
#     Overall Accuracy
                overall_accuracy = mir_eval.melody.overall_accuracy(ref_v, ref_c,est_v, est_c)
                output_arr.append([melody_extracted_renamed,voicing_recall_rate,voicing_false_alarm_rate,raw_pitch,raw_chroma,overall_accuracy])
                
                

                
                
                
                

                

In [91]:
df = pd.DataFrame(output_arr,columns=['File name','Voicing Recall Rate','Voicing False Alarm Rate','Raw Pitch Accuracy','Raw Chroma Accuracy','Overall Accuracy'])

In [94]:
voicing_recall = df['Voicing Recall Rate'].tolist()
voicing_false= df['Voicing False Alarm Rate'].tolist()
raw_pitch_acc = df['Raw Pitch Accuracy'].tolist()
raw_chroma_acc=df['Raw Chroma Accuracy'].tolist()
overall_acc=df['Overall Accuracy'].tolist()


In [96]:
average_voicing_recall = mean(voicing_recall)
average_voicing_false = mean(voicing_false)
average_raw_pitch_accuracy=mean(raw_pitch_acc)
average_raw_chroma_accuracy= mean(raw_chroma_acc)
average_overall_accuracy= mean(overall_acc)

In [100]:
results = [average_voicing_recall,average_voicing_false,average_raw_pitch_accuracy,average_raw_chroma_accuracy,average_overall_accuracy]

In [106]:
df = pd.DataFrame(results,index=['Avg Voicing Recall Rate','Avg Voicing False Alarm Rate','Avg Raw Pitch Accuracy','Avg Raw Chroma Accuracy','Avg Overall Accuracy'])

In [107]:
df

,0
Avg Voicing Recall Rate,0.755032
Avg Voicing False Alarm Rate,0.356327
Avg Raw Pitch Accuracy,0.494986
Avg Raw Chroma Accuracy,0.577532
Avg Overall Accuracy,0.546357
